1) What is inductive reasoning? Deductive reasoning? Give an example of each, different from the examples given in class.

Inductive Reasoning is a way to draw conclusions by going from specific information to general conclusion, like sickness, where you see specific 
symptoms like a cough, dry throat, or congestion, then you find a general solution like a common cold.

Deductive Reasoning is the opposite of inductive, moving from general information to a specific conclusion, usually in a A=B, B=C, so A=C format. For
example, Cats are cool, all cool things are cute, therefore all cats are cute. 

In [43]:
import pandas as pd
import numpy as np
import math
import statistics
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay, classification_report
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import precision_recall_curve, f1_score, auc

2) Preprocess your dataset. (I'm using the Credit Card.csv)

In [44]:
cc = pd.read_csv("Credit_card.csv")

In [45]:
#changing simple binary answers like gender to 1 and 2
def change_dt(x):
    xy = {'M':0,'F':1}
    cc['GENDER'] = cc['GENDER'].map(xy)

    co = {'Y':0,'N':1}
    cc['Car_Owner'] = cc['Car_Owner'].map(co)

    po = {'Y':0,'N':1}
    cc['Propert_Owner'] = cc['Propert_Owner'].map(po)
    

In [46]:
change_dt("Credit_card.csv")

In [47]:
cc = cc.drop('Type_Occupation', axis=1)
cc = cc.drop('EDUCATION', axis=1)
cc = cc.drop('Marital_status', axis=1)
cc = cc.drop('Housing_type', axis=1)
cc = cc.drop('Type_Income', axis=1)

3) Create a decision tree model tuned to the best of your abilities. Explain how you tuned it.

In [59]:
def dt(x):
    X = cc.drop('Car_Owner',axis=1)
    y = cc['Car_Owner']

    #split data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                        test_size=0.3,
                                                        random_state=42)
    
    model = tree.DecisionTreeClassifier(max_depth=7, random_state=42, min_impurity_decrease=0.01)
    
    model = model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    print(classification_report(y_test, y_pred))

    ConfusionMatrixDisplay.from_predictions(y_test, y_pred)

    tree.plot_tree(model, feature_names=list(X.columns),
               filled=True, fontsize=6, rounded=True)

    plt.show()


4) Create a random forest model tuned to the best of your abilities. Explain how you tuned it.

In [50]:
def rf(x):
    global X_test, X_train
    sc = StandardScaler()
    X_test = sc.fit_transform(X_test)
    X_train = sc.fit_transform(X_train)
    
    rf = RandomForestClassifier(n_estimators=100, random_state=42)
    rf = rf.fit(X_train, y_train)
    rf.score(X_test, y_test)

    print(rf.feature_importances_, X.columns)

    rf_tree = rf.estimators_[4]
    tree.plot_tree(rf_tree)
    plt.savefig("rf_tree.svg")

5) Create an xgboost model tuned to the best of your abilities. Explain how you tuned it.

In [52]:
def xgmodel1(x):
    xgb = XGBClassifier()
    xgb.fit(X_train, y_train)

    y_pred = xgb.predict(X_test)

    ConfusionMatrixDisplay.from_estimator(xgb, X_test, y_test)

    print(type(y_test))
    print(type(y_pred))
    fpr, tpr, thresholds = roc_curve(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred)
    print(fpr, tpr)
    auc

In [54]:
def xgmodel2(x):
    global recall, precision
    plt.plot(recall, precision, marker = ".")
    plt.xlabel("Recall")
    plt.ylabel("Precision")

    precision, recall, thresholds = precision_recall_curve(y_test, y_pred)
    pr_auc = auc(recall, precision)
    pr_auc

    plt.plot(recall, precision, marker = ".")
    plt.xlabel("Recall")
    plt.ylabel("Precision")

In [62]:
#Singular Value Decomposition
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression


X, y = make_classification(n_samples=100, n_features=20, n_informative=10, n_redundant=10, random_state=42)

# define the pipeline

steps = [('Credit_Card.csv', TruncatedSVD(n_components=5)), ('m', LogisticRegression())]
model = Pipeline(steps=steps)

# evaluate model

cc = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)

# report performance

print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Accuracy: 0.637 (0.120)


As I change and tweak the code it gets worse in some aspects and gets less reliable (changing the random state and n_samples), so the more I change the 
worse it gets for the most parts. 